# Make Classes for the Microplates in the Strateos catalog



In [1]:
import pandas as pd
from typing import List, Union, Optional, Tuple
import rdflib
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, OWL, XSD
from urllib.parse import quote


In [2]:
workbook = pd.ExcelFile('../owl/Strateos containers.xlsx',engine='openpyxl')
strateos_catalog = pd.read_excel(workbook, "Strateos Containers")
strateos_plates = strateos_catalog[strateos_catalog['Well Count'] > 1]
strateos_plates.head()


,Id,Vendor,Catalog Number,Name,Retired At,Well Count,Well Depth (mm),Height (mm),Well Volume (ul),Column Count,Capabilities,Acceptable Lids,Sale Price
0,384-pcr,Eppendorf,951020539,384-Well twin.tec PCR Plate,-,384,9.60,10.6,40.0,24,"bluewash, dispense-destination, echo_dest, env...","ultra-clear, foil",*$9.35*
1,96-10-spot-vplex-m-pro-inflamm1-MSD,Mesoscale,K15048G,96-well 10-spot v-plex mouse pro-inflammatory ...,-,96,11.68,14.35,410.0,12,"bluewash, cover, deseal, dispense-destination,...","low_evaporation, standard, universal, ultra-clear",*$0.00*
4,384-flat-white-white-nbs,Corning,3574,384-Well Low Flange White FlatBottom Polystyre...,-,384,11.43,14.35,80.0,24,"cover, deseal, dispense-destination, echo_dest...","universal, standard, ultra-clear, foil",*$11.80*
5,384-flat-white-white-optiplate,PerkinElmer,6007290,384-Well White Opaque Microplate,-,384,10.45,14.4,105.0,24,"bluewash, cover, dispense-destination, echo_de...","universal, ultra-clear, foil",*$8.28*
6,96-v-kf,Fisher,22-387-030,96-well KingFisher PCR microplate,-,96,12.90,14.5,200.0,12,"cover, dispense-destination, envision, image_p...",standard,*$6.31*


In [3]:
for i, x in strateos_plates.iterrows():
    break
print(x)
print(x.index)
x['Well Depth (mm)']

Id                                                            384-pcr
Vendor                                                      Eppendorf
Catalog Number                                              951020539
Name                                      384-Well twin.tec PCR Plate
Retired At                                                          -
Well Count                                                        384
Well Depth (mm)                                                   9.6
Height (mm)                                                      10.6
Well Volume (ul)                                                 40.0
Column Count                                                       24
Capabilities        bluewash, dispense-destination, echo_dest, env...
Acceptable Lids                                     ultra-clear, foil
Sale Price                                                    *$9.35*
Name: 0, dtype: object
Index(['Id', 'Vendor', 'Catalog Number', 'Name', 'Retired At', 'Wel

9.6

In [4]:
g = Graph(identifier='http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers')

In [5]:
co_ns = Namespace('http://www.semanticweb.org/rpg/ontologies/2021/5/container-ontology#')
strateos_ns = Namespace('http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#')
om_ns = Namespace('http://www.ontology-of-units-of-measure.org/resource/om-2/')

In [6]:
g.add((strateos_ns['384-pcr'], RDF.type, co_ns['Plate']))

In [7]:
RDF.type

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')

In [8]:
(strateos_ns['384-pcr'], RDF.type, co_ns['Plate']) in g

True

In [9]:
co_ns.Plate

rdflib.term.URIRef('http://www.semanticweb.org/rpg/ontologies/2021/5/container-ontology#Plate')

In [10]:

def add_vendor(vendor_string, graph=g) -> URIRef:
    vendor_iri = co_ns[quote(vendor_string)]
    vendor_name = Literal(vendor_string)    
    graph.add((vendor_iri, RDF.type, co_ns.VendorFirm))
    graph.add((vendor_iri, RDFS.label, vendor_name))
    graph.add((vendor_iri, co_ns.vendorName, vendor_name))
    return vendor_iri

def find_vendor(vendor_string, graph=g) -> Optional[URIRef]:
    vendor_iri = co_ns[quote(vendor_string)]
    if (vendor_iri, RDF.type, co_ns.VendorFirm) in graph:
        return vendor_iri
    return None

In [11]:
def catalog_entry(vendor: str, catalog_number: Union[int, str], graph=g) -> URIRef:
    if isinstance(catalog_number, int):
        catalog_number = str(catalog_number)
    assert isinstance(catalog_number, str)
    vendor_iri = find_vendor(vendor, graph)
    new_iri = strateos_ns[quote(vendor + catalog_number)]
    graph.add((new_iri, RDF.type, co_ns.CatalogEntry))
    graph.add((new_iri, co_ns.hasVendor, vendor_iri))
    graph.add((new_iri, co_ns.catalogNumber, Literal(catalog_number)))
    return new_iri
    

In [12]:
class WellDepth(URIRef):
    depth: float
    unit: URIRef
    count = 0
    def __new__(cls, *args, **kwargs):
        ns = kwargs['ns']
        URI = ns[f"wellDepth{cls.count}"]
        cls.count += 1
        return super().__new__(cls, URI)
    
    def __init__(self, depth: float, unit=om_ns.millimetre, ns=strateos_ns):
        self.depth = depth
        self.unit = unit
        super().__init__()
        
    def triples(self) -> List[Tuple[URIRef, URIRef, URIRef]]:
        return [
            (self, RDF.type, om_ns.Depth),
            (self, om_ns.hasNumericalValue, Literal(self.depth, datatype=XSD.float)),
            (self, om_ns.hasDimension, self.unit),
        ]
        

def make_well_depth(depth: float, unit=om_ns.millimetre, ns=strateos_ns, graph=g) -> URIRef:
    depth_uri = WellDepth(depth, unit=unit, ns=ns)
    for triple in depth_uri.triples():
        graph.add(triple)
    return depth_uri
    
    

In [13]:
class Height(URIRef):
    height: float
    unit: URIRef
    count = 0
    def __new__(cls, *args, **kwargs):
        ns = kwargs['ns']
        URI = ns[f"height{cls.count}"]
        cls.count += 1
        return super().__new__(cls, URI)
    
    def __init__(self, height: float, unit=om_ns.millimetre, ns=strateos_ns):
        self.height = height
        self.unit = unit
        super().__init__()
        
    def triples(self) -> List[Tuple[URIRef, URIRef, URIRef]]:
        return [
            (self, RDF.type, om_ns.Height),
            (self, om_ns.hasNumericalValue, Literal(self.height, datatype=XSD.float)),
            (self, om_ns.hasDimension, self.unit),
        ]
        

def make_height(height: float, unit=om_ns.millimetre, ns=strateos_ns, graph=g) -> URIRef:
    height_uri = Height(height, unit=unit, ns=ns)
    for triple in height_uri.triples():
        graph.add(triple)
    return height_uri
    
    

In [14]:
class WellVolume(URIRef):
    volume: float
    unit: URIRef
    count = 0
    def __new__(cls, *args, **kwargs):
        ns = kwargs['ns']
        URI = ns[f"wellvolume{cls.count}"]
        cls.count += 1
        return super().__new__(cls, URI)
    
    def __init__(self, volume: float, unit=om_ns.microlitre, ns=strateos_ns):
        self.volume = volume
        self.unit = unit
        super().__init__()
        
    def triples(self) -> List[Tuple[URIRef, URIRef, URIRef]]:
        return [
            (self, RDF.type, om_ns.Volume),
            (self, om_ns.hasNumericalValue, Literal(self.volume, datatype=XSD.float)),
            (self, om_ns.hasDimension, self.unit),
        ]
        

def make_well_volume(volume: float, unit=om_ns.microlitre, ns=strateos_ns, graph=g) -> URIRef:
    volume_uri = WellVolume(volume, unit=unit, ns=ns)
    for triple in volume_uri.triples():
        graph.add(triple)
    return volume_uri
    
    

In [15]:

for _, x in strateos_plates.iterrows():
    id = x['Id']
    vendor = x['Vendor']
    vendor_iri = find_vendor(vendor)
    if vendor_iri is None:
        vendor_iri = add_vendor(vendor)
    catalog_number = x['Catalog Number']
    plate_iri = rdflib.term.URIRef(strateos_ns[quote(vendor+id)])
    ce = catalog_entry(vendor, catalog_number)
    g.add((plate_iri, co_ns.hasCatalogEntry, ce))
    g.add((plate_iri, RDF.type, co_ns.Plate))
    g.add((plate_iri, co_ns.wellCount, Literal(int(x['Well Count']))))
    cols = int(x['Column Count'])
    rows = int(x['Well Count'] / cols)
    g.add((plate_iri, co_ns.columnCount, Literal(cols)))
    g.add((plate_iri, co_ns.rowCount, Literal(rows)))
    g.add((plate_iri, RDFS.comment, Literal(vendor + " " + x['Name'])))
    g.add((plate_iri, RDFS.label, Literal(id)))
    well_depth = make_well_depth(float(x['Well Depth (mm)']))
    g.add((plate_iri, co_ns.wellDepth, well_depth))
    well_vol = make_well_volume(float(x['Well Volume (ul)']))
    g.add((plate_iri, co_ns.wellVolume, well_vol))
    height = make_height(float(x['Height (mm)']))
    g.add((plate_iri, co_ns.height, height))
    print(f"Added {vendor} {id} as {plate_iri}")

Added Eppendorf 384-pcr as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Eppendorf384-pcr
Added Mesoscale 96-10-spot-vplex-m-pro-inflamm1-MSD as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Mesoscale96-10-spot-vplex-m-pro-inflamm1-MSD
Added Corning 384-flat-white-white-nbs as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Corning384-flat-white-white-nbs
Added PerkinElmer 384-flat-white-white-optiplate as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#PerkinElmer384-flat-white-white-optiplate
Added Fisher 96-v-kf as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Fisher96-v-kf
Added Chemspeed chemspeed-96-sealed-pin-rack as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Chemspeedchemspeed-96-sealed-pin-rack
Added Labcyte 384-echo as http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Labcyte384-echo
Added Costar 96-flat-clear-costar-3590 

In [16]:
g.serialize(format='turtle')

b'@prefix ns1: <http://www.ontology-of-units-of-measure.org/resource/om-2/> .\n@prefix ns2: <http://www.semanticweb.org/rpg/ontologies/2021/5/container-ontology#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#384-pcr> a ns2:Plate .\n\n<http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Chemspeedchemspeed-96-sealed-pin-rack> a ns2:Plate ;\n    rdfs:label "chemspeed-96-sealed-pin-rack" ;\n    ns2:columnCount 12 ;\n    ns2:hasCatalogEntry <http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#Chemspeedunknown> ;\n    ns2:height <http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#height5> ;\n    ns2:rowCount 8 ;\n    ns2:wellCount 96 ;\n    ns2:wellDepth <http://www.semanticweb.org/rpg/ontologies/2021/8/strateos-containers#wellDepth5> ;\n    ns2:wellVolume <http://www.semanticweb.org/rpg/ontologies/2021/

In [17]:
g.serialize('../owl/strateos-catalog.ttl', format='turtle')